In [1]:
import os
from tqdm import tqdm
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from numpy import asarray

In [2]:
longitud, altura = 224, 224
modelo = './Modelo_inception_exp3/modelo_tf_inception3-exp3.h5'
pesos_modelo = './Modelo_inception_exp3/pesos_tf_inception3-exp3.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
batch_normalization_94 (Batc (None, 2048)              8192      
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 5

In [7]:
def Dataset_loader(DIR, RESIZE):
  IMG = []
  read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
  for IMAGE_NAME in tqdm(os.listdir(DIR)):
    PATH = os.path.join(DIR, IMAGE_NAME)
    _, ftype = os.path.splitext(PATH)
    if (ftype == ".jpg" or ftype=='.jpeg'):
      img = read(PATH)
      img = cv2.resize(img, (RESIZE, RESIZE))
      IMG.append(np.array(img) / 255.)
  return IMG

In [8]:
def Dataset_loader2(DIR, RESIZE):
  IMG = []
  for IMAGE_NAME in tqdm(os.listdir(DIR)):
    PATH = os.path.join(DIR, IMAGE_NAME)
    _, ftype = os.path.splitext(PATH)
    if (ftype == ".jpg" or ftype=='.jpeg'):
      image = Image.open(PATH).resize((RESIZE, RESIZE))
      data = asarray(image)
      IMG.append(np.array(data) / 255.)
  return IMG

In [5]:
train_benign = np.array(Dataset_loader('./DermMel/train_sep/NotMelanoma', altura))
train_malign = np.array(Dataset_loader('./DermMel/train_sep/Melanoma', altura))


benign_train_label = np.zeros(len(train_benign))
malign_train_label = np.ones(len(train_malign))

X_train = np.concatenate((train_benign, train_malign), axis = 0)
Y_train = np.concatenate((benign_train_label, malign_train_label), axis = 0)

100%|██████████| 5341/5341 [00:14<00:00, 366.40it/s]


In [6]:
y_pred = cnn.predict(X_train)
print("Done.")

Done.


In [7]:
#Segundo conjunto de imágenes
train_benign2 = np.array(Dataset_loader('./input/skin-cancer-malignant-vs-benign/train/benign', altura))
train_malign2 = np.array(Dataset_loader('./input/skin-cancer-malignant-vs-benign/train/malignant', altura))


benign_train_label2 = np.zeros(len(train_benign2))
malign_train_label2 = np.ones(len(train_malign2))

X_train2 = np.concatenate((train_benign2, train_malign2), axis = 0)
Y_train2 = np.concatenate((benign_train_label2, malign_train_label2), axis = 0)

100%|██████████| 1197/1197 [00:03<00:00, 398.78it/s]


In [8]:
y_pred4 = cnn.predict(X_train2)
print("Done.")

Done.


In [9]:
def ValidationTraining(y_pred,Y_test):
  print(len(y_pred))
  for j in range(50,0,-2):
      predictions = []
      truePositives = 0
      falsePositives = 0
      trueNegatives = 0
      falseNegatives = 0
      for i in range(len(y_pred)):
        if y_pred[i][0] > j/100:
          predictions.append(1)
          if Y_test[i]==1:
              truePositives+=1
          else: falsePositives+=1
        else:
          predictions.append(0)
          if Y_test[i]==0:
              trueNegatives+=1
          else: falseNegatives+=1

      sensitivity=truePositives/(truePositives+falseNegatives)
      especificity=trueNegatives/(trueNegatives+falsePositives)
      accuracy=accuracy_score(Y_test, predictions)
      print("ClassificationNumber:",j/100)
      print("Accuracy model: ",accuracy)
      print("Sensitivity: ",sensitivity)
      print("Especificity: ",especificity)
      print("TP",truePositives,"FN",falseNegatives,"TN",trueNegatives,"FP",falsePositives)
      print("-----------------------------------------------------------------")

In [10]:
ValidationTraining(y_pred,Y_train)

10682
ClassificationNumber: 0.5
Accuracy model:  0.8946826436996818
Sensitivity:  0.8743680958621981
Especificity:  0.9149971915371653
TP 4670 FN 671 TN 4887 FP 454
-----------------------------------------------------------------
ClassificationNumber: 0.48
Accuracy model:  0.8960868751170192
Sensitivity:  0.8839168695000936
Especificity:  0.908256880733945
TP 4721 FN 620 TN 4851 FP 490
-----------------------------------------------------------------
ClassificationNumber: 0.46
Accuracy model:  0.8970230293952444
Sensitivity:  0.891031642014604
Especificity:  0.9030144167758847
TP 4759 FN 582 TN 4823 FP 518
-----------------------------------------------------------------
ClassificationNumber: 0.44
Accuracy model:  0.8967421831117768
Sensitivity:  0.8977719528178244
Especificity:  0.8957124134057293
TP 4795 FN 546 TN 4784 FP 557
-----------------------------------------------------------------
ClassificationNumber: 0.42
Accuracy model:  0.8979591836734694
Sensitivity:  0.90619734132184

In [11]:
ValidationTraining(y_pred4,Y_train2)

2637
ClassificationNumber: 0.5
Accuracy model:  0.6977626090254077
Sensitivity:  0.3893065998329156
Especificity:  0.9541666666666667
TP 466 FN 731 TN 1374 FP 66
-----------------------------------------------------------------
ClassificationNumber: 0.48
Accuracy model:  0.701554797117937
Sensitivity:  0.4018379281537176
Especificity:  0.9506944444444444
TP 481 FN 716 TN 1369 FP 71
-----------------------------------------------------------------
ClassificationNumber: 0.46
Accuracy model:  0.7087599544937428
Sensitivity:  0.4177109440267335
Especificity:  0.9506944444444444
TP 500 FN 697 TN 1369 FP 71
-----------------------------------------------------------------
ClassificationNumber: 0.44
Accuracy model:  0.7133105802047781
Sensitivity:  0.43358395989974935
Especificity:  0.9458333333333333
TP 519 FN 678 TN 1362 FP 78
-----------------------------------------------------------------
ClassificationNumber: 0.42
Accuracy model:  0.7174819871065605
Sensitivity:  0.4469507101086048
Espe

In [12]:
#TESTING

In [13]:
#Para este modelo se ha encontrado que el mejor resultado de corte es 0.08
def ValidationTesting(y_pred,Y_test):
  print(y_pred[20],Y_test[20])
  classificationNumber=0.05
  predictions = []
  truePositives = 0
  falsePositives = 0
  trueNegatives = 0
  falseNegatives = 0
  for i in range(len(y_pred)):
    if y_pred[i][0] > classificationNumber:
      predictions.append(1)
      if Y_test[i]==1:
          truePositives+=1
      else: falsePositives+=1
    else:
      predictions.append(0)
      if Y_test[i]==0:
          trueNegatives+=1
      else: falseNegatives+=1

  sensitivity=truePositives/(truePositives+falseNegatives)
  especificity=trueNegatives/(trueNegatives+falsePositives)
  accuracy=accuracy_score(Y_test, predictions)
  print("ClassificationNumber:",classificationNumber)
  print("Accuracy model: ",accuracy)
  print("Sensitivity: ",sensitivity)
  print("Especificity: ",especificity)
  print("TP",truePositives,"FN",falseNegatives,"TN",trueNegatives,"FP",falsePositives)
  print("-----------------------------------------------------------------")

In [9]:
import random
random.seed(30)

test_benign = np.array(Dataset_loader('./DermMel/test/NotMelanoma', altura))
test_malign = np.array(Dataset_loader('./DermMel/test/Melanoma', altura))


benign_test_label = np.zeros(len(test_benign))
malign_test_label = np.ones(len(test_malign))

X_test = np.concatenate((test_benign, test_malign), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

random.shuffle(X_test)
random.shuffle(Y_test)

X_experimentsArray=np.array_split(X_test, 13)
Y_experimentsArray=np.array_split(Y_test, 13)

X_13E=[]
Y_13E=[]

print(len(test_benign),len(test_malign))
rangeNumber=0
for i in range(0,13):
    #100 imagenes benignas
    benign=(test_benign[rangeNumber:rangeNumber+100])
    zeros=(np.zeros(100))
    #100 imagenes malignas
    malign=(test_malign[rangeNumber:rangeNumber+100])
    ones=(np.ones(100))
    
    rangeNumber+=100
    print(rangeNumber)
    X_13E.append(np.concatenate((benign,malign),axis=0))
    Y_13E.append(np.concatenate((zeros,ones),axis=0))  

100%|██████████| 1781/1781 [00:04<00:00, 365.69it/s]


1780 1781
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [7]:
#Predicción (solo se ejecuta una vez)
y_pred2 = cnn.predict(X_test)
print("Done.")

Done.


In [8]:
print(len(X_test),len(Y_test),len(malign_test_label),len(benign_test_label),len(test_benign),len(test_malign))

3561 3561 1781 1780 1780 1781


In [9]:
ValidationTesting(y_pred2,Y_test)

ClassificationNumber: 0.05
Accuracy model:  0.7604605447907891
Sensitivity:  0.9893318360471645
Especificity:  0.5314606741573034
TP 1762 FN 19 TN 946 FP 834
-----------------------------------------------------------------


In [10]:
predictionsArray=[]
for experiment in X_13E:
   predictionsArray.append(cnn.predict(experiment)) 

In [14]:
print(len(X_13E),len(Y_13E),len(predictionsArray))
for i in range(len(predictionsArray)):
    print("Experimento ",i)
    ValidationTesting(predictionsArray[i],Y_13E[i])
    print("---------------------------------------------")

13 13 13
Experimento  0
[0.02075209 0.9932214 ] 0.0
ClassificationNumber: 0.05
Accuracy model:  0.795
Sensitivity:  1.0
Especificity:  0.59
TP 100 FN 0 TN 59 FP 41
-----------------------------------------------------------------
---------------------------------------------
Experimento  1
[0.05011538 0.9969584 ] 0.0
ClassificationNumber: 0.05
Accuracy model:  0.74
Sensitivity:  0.98
Especificity:  0.5
TP 98 FN 2 TN 50 FP 50
-----------------------------------------------------------------
---------------------------------------------
Experimento  2
[0.00412178 0.9142126 ] 0.0
ClassificationNumber: 0.05
Accuracy model:  0.77
Sensitivity:  0.99
Especificity:  0.55
TP 99 FN 1 TN 55 FP 45
-----------------------------------------------------------------
---------------------------------------------
Experimento  3
[0.32731795 0.99840194] 0.0
ClassificationNumber: 0.05
Accuracy model:  0.81
Sensitivity:  1.0
Especificity:  0.62
TP 100 FN 0 TN 62 FP 38
---------------------------------------

In [18]:
#Segundo dataset de testing
import random
random.seed(30)

test_benign2 = np.array(Dataset_loader2('./input/skin-cancer-malignant-vs-benign/test/benign', altura))
test_malign2 = np.array(Dataset_loader2('./input/skin-cancer-malignant-vs-benign/test/malignant', altura))

benign_test_label2 = np.zeros(len(test_benign2))
malign_test_label2 = np.ones(len(test_malign2))

X_test2 = np.concatenate((test_benign2, test_malign2), axis = 0)
Y_test2 = np.concatenate((benign_test_label2, malign_test_label2), axis = 0)

X_13E_2=[]
Y_13E_2=[]

print(len(test_malign2),len(test_benign2))
rangeNumber=0
for i in range(0,13):
    #20 imagenes benignas
    benign=(test_benign2[rangeNumber:rangeNumber+20])
    zeros=(np.zeros(20))
    #20 imagenes malignas
    malign=(test_malign2[rangeNumber:rangeNumber+20])
    ones=(np.ones(20))
    
    rangeNumber+=20
    print(rangeNumber)
    X_13E_2.append(np.concatenate((benign,malign),axis=0))
    Y_13E_2.append(np.concatenate((zeros,ones),axis=0))  

100%|██████████| 300/300 [00:00<00:00, 355.56it/s]


300 360
20
40
60
80
100
120
140
160
180
200
220
240
260


In [ ]:
y_pred3 = cnn.predict(X_test2)
print("Done.")

In [ ]:
ValidationTesting(y_pred3,Y_test2)

In [19]:
predictionsArray2=[]
for experiment in X_13E_2:
   predictionsArray2.append(cnn.predict(experiment)) 

In [20]:
print(len(X_13E_2[1]),len(Y_13E_2[1]),len(predictionsArray2[1]))
for i in range(len(predictionsArray2)):
    print("Experimento ",i)
    print(len(predictionsArray2[i]),len(Y_13E_2[i]))
    ValidationTesting(predictionsArray2[i],Y_13E_2[i])
    print("---------------------------------------------")

40 40 40
Experimento  0
40 40
[0.7490658  0.61813766] 1.0
ClassificationNumber: 0.05
Accuracy model:  0.8
Sensitivity:  0.95
Especificity:  0.65
TP 19 FN 1 TN 13 FP 7
-----------------------------------------------------------------
---------------------------------------------
Experimento  1
40 40
[0.6652078 0.9931225] 1.0
ClassificationNumber: 0.05
Accuracy model:  0.775
Sensitivity:  0.95
Especificity:  0.6
TP 19 FN 1 TN 12 FP 8
-----------------------------------------------------------------
---------------------------------------------
Experimento  2
40 40
[0.08764985 0.99885404] 1.0
ClassificationNumber: 0.05
Accuracy model:  0.75
Sensitivity:  0.95
Especificity:  0.55
TP 19 FN 1 TN 11 FP 9
-----------------------------------------------------------------
---------------------------------------------
Experimento  3
40 40
[0.97480994 0.96431935] 1.0
ClassificationNumber: 0.05
Accuracy model:  0.85
Sensitivity:  0.95
Especificity:  0.75
TP 19 FN 1 TN 15 FP 5
----------------------